In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import RegexpTokenizer as regextoken
from nltk.corpus import stopwords
from nltk import FreqDist, bigrams, trigrams,ngrams
from nltk import WordNetLemmatizer
import matplotlib
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
# import gensim
# from gensim.models.keyedvectors import KeyedVectors
from keras.models import Sequential
from keras.models import Model
from keras.layers import Dense, Flatten, Embedding, Conv1D, MaxPooling1D, Dropout, LSTM, GRU
from keras.regularizers import l1, l2
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings("ignore")
import collections
zomato = pd.read_csv("zomato.csv", na_values = ["-", ""])
# Making a copy of the data to work on
data = zomato.copy()
data.drop(['url','address','menu_item','phone','location'],axis=1,inplace=True)
data.rename(columns={'listed_in(city)':'location'}, inplace=True)
data=data[data['rate'].notnull()]

Using TensorFlow backend.


In [2]:
data['dish_liked']= data["dish_liked"].astype(str)

In [3]:
sam=data[['dish_liked','location']].groupby(['location'],as_index=False).agg(lambda x:' '.join(list(x)))

In [4]:
pair={}
for i in range(0,30):
    s=sam[sam.index==i]
    s["dish_liked"]= s["dish_liked"].apply(lambda x: x.lower())
    tokenizer = regextoken("[a-zA-Z]+") 
    dish_tokens = s["dish_liked"].apply(tokenizer.tokenize)
    stop = stopwords.words("english")
    stop.extend(["rated", "n", "nan", "x"])
    dish_tokens = dish_tokens.apply(lambda x: [token for token in x if token not in stop])
    all_dishes = dish_tokens.astype(str).str.cat()
    cleaned_dishes = tokenizer.tokenize(all_dishes)
    j=s['location'].values[0]
    pair[j] = cleaned_dishes

In [5]:
dish2={}
dish3={}

In [6]:
for i in pair.keys():
    trigram = lambda a: zip(a, a[1:],a[2:])
    ap=list(trigram(pair[i]))
    trigramFreq = collections.Counter(ap)
    dish3[i]=trigramFreq.most_common(10)  

In [8]:
for i in pair.keys():    
    bigram = lambda a: zip(a, a[1:])
    apw=list(bigram(pair[i]))
    bigramFreq = collections.Counter(apw)
    dish2[i]=bigramFreq.most_common(10)

In [7]:
print(dish3['BTM'])

[(('peri', 'peri', 'chicken'), 47), (('panneer', 'butter', 'masala'), 31), (('murgh', 'ghee', 'roast'), 21), (('biryani', 'mutton', 'biryani'), 16), (('biryani', 'tandoori', 'chicken'), 15), (('chicken', 'biryani', 'chicken'), 13), (('honey', 'chilli', 'potatoes'), 13), (('chicken', 'boneless', 'biryani'), 13), (('long', 'island', 'iced'), 12), (('island', 'iced', 'tea'), 12)]


In [8]:
print(dish3['Banashankari'])

[(('panneer', 'butter', 'masala'), 20), (('hot', 'chocolate', 'fudge'), 9), (('masala', 'dosa', 'idli'), 9), (('masala', 'papad', 'paneer'), 7), (('coffee', 'masala', 'dosa'), 6), (('biryani', 'chicken', 'biryani'), 5), (('churros', 'cannelloni', 'minestrone'), 5), (('cannelloni', 'minestrone', 'soup'), 5), (('minestrone', 'soup', 'hot'), 5), (('soup', 'hot', 'chocolate'), 5)]


In [9]:
print(dish3['Church Street'])

[(('peri', 'peri', 'chicken'), 38), (('long', 'island', 'iced'), 16), (('island', 'iced', 'tea'), 16), (('murgh', 'ghee', 'roast'), 15), (('vanilla', 'ice', 'cream'), 11), (('biryani', 'mutton', 'biryani'), 11), (('panneer', 'butter', 'masala'), 11), (('tom', 'yum', 'soup'), 10), (('hot', 'chocolate', 'fudge'), 10), (('cocktails', 'beef', 'steak'), 10)]


In [10]:
print(dish3['Residency Road'])

[(('peri', 'peri', 'chicken'), 29), (('murgh', 'ghee', 'roast'), 14), (('biryani', 'mutton', 'biryani'), 12), (('vanilla', 'ice', 'cream'), 10), (('hot', 'chocolate', 'fudge'), 9), (('long', 'island', 'iced'), 9), (('island', 'iced', 'tea'), 9), (('panneer', 'butter', 'masala'), 9), (('biryani', 'dal', 'makhani'), 8), (('bbq', 'chicken', 'pizza'), 8)]


In [11]:
print(dish3['MG Road'])

[(('peri', 'peri', 'chicken'), 39), (('murgh', 'ghee', 'roast'), 20), (('biryani', 'mutton', 'biryani'), 13), (('long', 'island', 'iced'), 12), (('island', 'iced', 'tea'), 12), (('cocktails', 'beef', 'steak'), 12), (('vanilla', 'ice', 'cream'), 11), (('panneer', 'butter', 'masala'), 11), (('biryani', 'chicken', 'biryani'), 10), (('biryani', 'dal', 'makhani'), 9)]


In [39]:
print(dish3['Old Airport Road'])

[(('peri', 'peri', 'chicken'), 28), (('biryani', 'mutton', 'biryani'), 18), (('murgh', 'ghee', 'roast'), 18), (('panneer', 'butter', 'masala'), 18), (('sweet', 'corn', 'soup'), 14), (('long', 'island', 'iced'), 12), (('island', 'iced', 'tea'), 12), (('hot', 'chocolate', 'chocolate'), 12), (('peri', 'peri', 'fries'), 12), (('chicken', 'boneless', 'biryani'), 12)]


In [9]:
pair.keys()

dict_keys(['BTM', 'Banashankari', 'Bannerghatta Road', 'Basavanagudi', 'Bellandur', 'Brigade Road', 'Brookefield', 'Church Street', 'Electronic City', 'Frazer Town', 'HSR', 'Indiranagar', 'JP Nagar', 'Jayanagar', 'Kalyan Nagar', 'Kammanahalli', 'Koramangala 4th Block', 'Koramangala 5th Block', 'Koramangala 6th Block', 'Koramangala 7th Block', 'Lavelle Road', 'MG Road', 'Malleshwaram', 'Marathahalli', 'New BEL Road', 'Old Airport Road', 'Rajajinagar', 'Residency Road', 'Sarjapur Road', 'Whitefield'])